## Important Dependencies with their required versions


In [ ]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# !pip install text_generation

## Imports and Setup and Initializing Agent

In [1]:
# # to auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [ ]:
# from langchain_community.llms import HuggingFaceEndpoint, HuggingFaceHub
# import os
# from dotenv import load_dotenv
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate

# question = "Who won the FIFA World Cup in the year 1994? "

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = PromptTemplate.from_template(template)

# repo_id = "bigcode/starcoder2-7b"
# load_dotenv()
# # llm = HuggingFaceEndpoint(
# #     repo_id=repo_id, max_new_tokens=12000, temperature=0.5, token= os.environ.get("HUGGINGFACE_TOKEN")
# # )

# llm = HuggingFaceHub(
#         repo_id=repo_id,
#         huggingfacehub_api_token= os.environ.get("HUGGINGFACE_TOKEN"),
#         task="text-generation",
#         model_kwargs={
#             "temperature": 0.1,  # can't be 0
#             "max_new_tokens": 12000,
#         },
#         cache=False,
#     )

# llm_chain = LLMChain(prompt=prompt, llm=llm)
# print(llm_chain.run({"question":question}))

In [3]:
# from Imports import *
%load_ext autoreload
%autoreload 2
try:
    from Configuration import *
    from TestGenerator import *
    from TestFix import *

    llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"],repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1')
    t = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
    f = TestFix(UnitTestFeedbackChain,True,globals())
except Exception as e:
    print(e)
    exit(-1)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TestFix.__init__() takes 4 positional arguments but 5 were given


## Generate Unit Tests from Code

In [ ]:
t.generate()

In [2]:
f.generate()

NameError: name 'f' is not defined